In [282]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Import output

In [283]:
sp_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/statepoint.50.h5'
sm_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/summary.h5'
sm=openmc.Summary(sm_file)
sp = openmc.StatePoint(sp_file)
sp.link_with_summary(sm)
model=openmc.model.Model
model_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/model_2B_72_families_power.xml'
model=openmc.model.Model.from_model_xml(model_file)

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/statepoint.py:671: RuntimeWarning: A Summary object has already been linked.
  warnings.warn('A Summary object has already been linked.',


### Get Power and Flux values

In [284]:
for index, tally in enumerate(sp.tallies):
    if index==0:
        t_power=sp.tallies[tally]
    if index==1:
        t_flux=sp.tallies[tally]
    if index==2:
        t_fission=sp.tallies[tally]

def tally_cell(cell,x,y): 
    A=[]
    cell_index = 99
    cell_id=99
    i=0
    for all_cell in model.geometry.get_all_material_cells().values():
        i=i+1
        if all_cell.name==cell+'_'+str(x)+'_'+str(y):
            cell_index=all_cell.id
            cell_id=i-1
    tally_data_reshaped =t_power.get_reshaped_data()
    power=tally_data_reshaped[cell_id]
    if x==8 or y==8:
        A.append(2*power[0][0])
    else : 
        A.append(power[0][0])
    tally_data_reshaped =t_flux.get_reshaped_data()
    flux=tally_data_reshaped[cell_id]
    A.append([flux[0][0][0],flux[1][0][0]])

    tally_data_reshaped =t_fission.get_reshaped_data()
    fission=tally_data_reshaped[cell_id]
    if x==8 or y==8:
        A.append(2*fission[0][0])
    else : 
        A.append(fission[0][0])
    return(A)

def power_cell(cell,x,y):
    return(tally_cell(cell,x,y)[0])

def flux_cell(cell,x,y):
    return(tally_cell(cell,x,y)[1])

def fission_cell(cell,x,y):
    return(tally_cell(cell,x,y)[2])



In [285]:
print(tally_cell('fuel',7,8))

print(power_cell('fuel',7,8))
print(flux_cell('fuel',7,8))
print(fission_cell('fuel',7,8))

[1328550.4921856122, [0.020019177060316078, 0.05743978575609926], 0.007545751011476627]
1328550.4921856122
[0.020019177060316078, 0.05743978575609926]
0.007545751011476627


### Mapping geom

In [286]:
def flux_cell(cell,x,y):    
    cell_index = 99
    cell_id=99
    i=0
    for all_cell in model.geometry.get_all_material_cells().values():
        i=i+1
        if all_cell.name==cell+'_'+str(x)+'_'+str(y):
            cell_index=all_cell.id
            cell_id=i-1
    tally_data_reshaped =t_flux.get_reshaped_data()
    flux=tally_data_reshaped[cell_id]
    flux_return=[flux[0][0][0],flux[1][0][0]]
    return(flux_return)

In [287]:
GT=[(5,2),(8,2),(3,3),(2,5),(5,5),(8,5),(2,8),(5,8)]
IT=[(8,8)]

### Mapping benchmark

In [288]:
A2=[(0,0,0.9476),(1,0,0.9418),(2,0,0.9458),(3,0,0.9551),(4,0,0.9645),(5,0,0.9739),(6,0,0.9714),(7,0,0.9721),(8,0,0.9764),(1,1,0.9389),(2,1,0.9483 ),(3,1,0.9649 ),(4,1,0.9837),(5,1,1.0115),(6,1,0.9880),(7,1,0.9880),(8,1,1.0122),(2,2,0.9736),(3,2,1.0180),(4,2,1.0360),(6,2,1.0281),(7,2,1.0266),(4,3,1.0516),(5,3,1.0458),(6,3,1.0089),(7,3,1.0053),(8,3,1.0328),(4,4,1.0313),(5,4,1.0451),(6,4,1.0118),(7,4,1.0089),(8,4,1.0353),(6,5,1.0386),(7,5,1.0368),(6,6,1.0104),(7,6,1.0093 ),(8,6,1.0371),(7,7,1.0089),(8,7,1.0364)]
B2=[(0,0,0.9478),(1,0,0.9435),(2,0,0.9475),(3,0,0.9565),(4,0,0.9662),(5,0,0.9752 ),(6,0,0.9738),(7,0,0.9742),(8,0,0.9781),(1,1,0.9406),(2,1,0.9503 ),(3,1,0.9666 ),(4,1,0.9839),(5,1,1.0099),(6,1,0.9893),(7,1,0.9893),(8,1,1.0103 ),(2,2,0.9745),(3,2, 1.0146),(4,2,1.0326),(6,2,1.0258),(7,2,1.0243),(4,3,1.0474),(5,3,1.0427),(6,3,1.0095),(7,3,1.0066),(8,3,1.0308),(4,4,  1.0308 ),(5,4,1.0424),(6,4,1.0124),(7,4,1.0099),(8,4,1.0337),(6,5,1.0373),(7,5,1.0359),(6,6,1.0121),(7,6,1.0113),(8,6,1.0362),(7,7,1.0113),(8,7,1.0355)]

### Create Power RD list

In [292]:
rd_list=[]
#Create power lis
for pos in A2:
    result=power_cell('fuel',pos[0],pos[1])
    bench=pos[2]
    rd=100*(result-bench)#/bench
    rd_list.append([pos[0],pos[1],result,bench,0])
    
#Sum
sum_power=0
N=0
for k in range((len(rd_list))):
    sum_power=sum_power+rd_list[k][2]
    N=N+1
    
#Normalisation 
avg_power=sum_power/N
for k in range((len(rd_list))):
    rd_list[k][2]=rd_list[k][2]/avg_power

#Calcul rd
for pos in rd_list:
    pos[4]=100*(pos[2]-pos[3])/pos[3]
    
#Print
for k in range((len(rd_list))):
    print(rd_list[k])

[0, 0, 0.9478910493224066, 0.9476, 0.030714364964821085]
[1, 0, 0.9457434260899411, 0.9418, 0.41871162560428304]
[2, 0, 0.9555066171759131, 0.9458, 1.0262864427905631]
[3, 0, 0.9592104828686447, 0.9551, 0.4303719891785985]
[4, 0, 0.9709816689577269, 0.9645, 0.672023738489045]
[5, 0, 0.9672396216222702, 0.9739, -0.6838872962039033]
[6, 0, 0.9771131866153702, 0.9714, 0.5881394498013305]
[7, 0, 0.9724670233153972, 0.9721, 0.03775571601659147]
[8, 0, 0.9815332244175043, 0.9764, 0.5257296617681586]
[1, 1, 0.9456637920922389, 0.9389, 0.7203953660921275]
[2, 1, 0.9432796705035826, 0.9483, -0.5294030893617417]
[3, 1, 0.9686364704109588, 0.9649, 0.38723913472471905]
[4, 1, 0.9886823846165708, 0.9837, 0.5064943190577152]
[5, 1, 1.007980487635489, 1.0115, -0.3479498135947692]
[6, 1, 0.9808799191059151, 0.988, -0.7206559609397662]
[7, 1, 0.9841211219999728, 0.988, -0.39259898785700154]
[8, 1, 1.0030367972386842, 1.0122, -0.9052759100292205]
[2, 2, 0.9725788315153503, 0.9736, -0.10488583449565797]


### Verify Power Normalisation

In [293]:
test=0
N=0
for A in rd_list:
    test=test+A[2]
    N=N+1
print(test/N) 

1.0000000000000004


### Maximum rd

In [305]:
rd_max=0
x=99
y=99
for A in rd_list:
    if abs(A[4])>abs(rd_max):
        rd_max=A[4]
        x=A[0]
        y=A[1]
print(x,y,rd_max)   

8 3 -1.2194013270145
